# About this Notebook

The aim is to conduct data checking and data cleaning, primarily focusing on:
- Missing Data
- Duplicate Data
- Data Values
- Data Types

Given the large number of features we are handling, the features are grouped into the following for detailed checking:<br>
- Part 1. Flat
- Part 2. Transaction
- Part 3. Block
- Part 4. Location
- Part 5. Facilities
    - Hawker & Malls
    - Schools
    - Transport
    - Others


# Imports and Function creation

### Libraries imports

In [1]:
import numpy as np 
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


#Visualisation:
import seaborn               as sns
import matplotlib.pyplot     as plt
sns.set_theme(style="whitegrid")

### Function Creation

In [2]:
#Showing missing, duplicates, shape, dtypes
def df_summary(df):
    print(f"Shape(col,rows): {df.shape}")
    print(f"Number of duplicates: {df.duplicated().sum()}")
    print('---'*20)
    print(f'Number of each unqiue datatypes:\n{df.dtypes.value_counts()}')
    print('---'*20)
    print("Columns with missing values:")
    isnull_df = pd.DataFrame(df.isnull().sum()).reset_index()
    isnull_df.columns = ['col','num_nulls']
    isnull_df['perc_null'] = ((isnull_df['num_nulls'])/(len(df))).round(2)
    _df_ = isnull_df[isnull_df['num_nulls']>0]
    if _df_.empty:
        print("--No Missing Data--")
    else:
        print(_df_)
        
#Showing details of columns containing missing data
def df_missing(df):
    isnull_df = pd.DataFrame(df.isnull().sum()).reset_index()
    isnull_df.columns = ['col','num_nulls']
    isnull_df['perc_null'] = ((isnull_df['num_nulls'])/(len(df))).round(2)
    isnull_df = isnull_df[isnull_df['num_nulls']>0]
    isnull_df.reset_index(drop = True,inplace=True)
    
    if isnull_df.empty:
        print("--No Missing Data--")
    else:
        return isnull_df

#Showing min and max values of each column:
def get_min_max_df(df):
    min_max_list = []
    _df = df.select_dtypes(exclude=['object'])
    for col in _df.columns:
        min_val = df[col].min()
        max_val = df[col].max()
        min_max_list.append([col, min_val, max_val])
    min_max_df = pd.DataFrame(min_max_list, columns=['Column Name', 'Minimum Value', 'Maximum Value'])
    return min_max_df


### Import Data

In [4]:
df=pd.read_csv("../data_input/train.csv",low_memory=False)
df.head()
df_summary(df)

,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
0,88471,2016-05,KALLANG/WHAMPOA,4 ROOM,3B,UPP BOON KENG RD,10 TO 12,90.0,Model A,2006,...,78,1,1.317659,103.882504,1138.633422,Geylang Methodist School,224,0,1.317659,103.882504
1,122598,2012-07,BISHAN,5 ROOM,153,BISHAN ST 13,07 TO 09,130.0,Improved,1987,...,45,1,1.349783,103.854529,447.894399,Kuo Chuan Presbyterian Secondary School,232,0,1.350110,103.854892
2,170897,2013-07,BUKIT BATOK,EXECUTIVE,289B,BT BATOK ST 25,13 TO 15,144.0,Apartment,1997,...,39,0,1.345245,103.756265,180.074558,Yusof Ishak Secondary School,188,0,1.342334,103.760013
3,86070,2012-04,BISHAN,4 ROOM,232,BISHAN ST 22,01 TO 05,103.0,Model A,1992,...,20,1,1.354789,103.844934,389.515528,Catholic High School,253,1,1.354789,103.844934
4,153632,2017-12,YISHUN,4 ROOM,876,YISHUN ST 81,01 TO 03,83.0,Simplified,1987,...,74,0,1.416280,103.838798,312.025435,Orchid Park Secondary School,208,0,1.414888,103.838335


Shape(col,rows): (150634, 77)
Number of duplicates: 0
------------------------------------------------------------
Number of each unqiue datatypes:
int64      32
float64    25
object     20
dtype: int64
------------------------------------------------------------
Columns with missing values:
                      col  num_nulls  perc_null
45  Mall_Nearest_Distance        829       0.01
46       Mall_Within_500m      92789       0.62
47        Mall_Within_1km      25426       0.17
48        Mall_Within_2km       1940       0.01
50     Hawker_Within_500m      97390       0.65
51      Hawker_Within_1km      60868       0.40
52      Hawker_Within_2km      29202       0.19


**Comments**
- There is 77 features
- No Duplication
- Need to Address Missing Data (found under features columns related to Hawker and Mall)
- Need to check datatypes
- Need to rename some columns for clarity

# Part 1: Features: Unit/Flat <a class="anchor" id="Flat"></a>
- Unit storey
- Flat Model & Type
- Floor area



In [5]:
Flat_col = [
    'storey_range',
    'mid_storey',
    'lower',
    'upper',
    'mid',
    'full_flat_type',
    'flat_model',
    'flat_type',
    'floor_area_sqm',
    'floor_area_sqft']

In [6]:
df.loc[:5,Flat_col]

,storey_range,mid_storey,lower,upper,mid,full_flat_type,flat_model,flat_type,floor_area_sqm,floor_area_sqft
0,10 TO 12,11,10,12,11,4 ROOM Model A,Model A,4 ROOM,90.0,968.760
1,07 TO 09,8,7,9,8,5 ROOM Improved,Improved,5 ROOM,130.0,1399.320
2,13 TO 15,14,13,15,14,EXECUTIVE Apartment,Apartment,EXECUTIVE,144.0,1550.016
3,01 TO 05,3,1,5,3,4 ROOM Model A,Model A,4 ROOM,103.0,1108.692
4,01 TO 03,2,1,3,2,4 ROOM Simplified,Simplified,4 ROOM,83.0,893.412
5,07 TO 09,8,7,9,8,3 ROOM Improved,Improved,3 ROOM,65.0,699.660


## 1.1 Data types and Value check

In [7]:
#Check datatype
df.loc[:,Flat_col].dtypes

storey_range        object
mid_storey           int64
lower                int64
upper                int64
mid                  int64
full_flat_type      object
flat_model          object
flat_type           object
floor_area_sqm     float64
floor_area_sqft    float64
dtype: object

In [8]:
#checking numerical data values
get_min_max_df(df.loc[:,Flat_col])

,Column Name,Minimum Value,Maximum Value
0,mid_storey,2.000,50.00
1,lower,1.000,49.00
2,upper,3.000,51.00
3,mid,2.000,50.00
4,floor_area_sqm,31.000,280.00
5,floor_area_sqft,333.684,3013.92


In [9]:
#Checking non-numerical data values
_ls = list(df.loc[:,Flat_col].select_dtypes(include=['object']).columns)
for i in _ls:
    _a = np.sort(df[i].unique())
    print(f"\nColumns: {i} \n Unique values: {_a}")


Columns: storey_range 
 Unique values: ['01 TO 03' '01 TO 05' '04 TO 06' '06 TO 10' '07 TO 09' '10 TO 12'
 '11 TO 15' '13 TO 15' '16 TO 18' '16 TO 20' '19 TO 21' '21 TO 25'
 '22 TO 24' '25 TO 27' '26 TO 30' '28 TO 30' '31 TO 33' '31 TO 35'
 '34 TO 36' '36 TO 40' '37 TO 39' '40 TO 42' '43 TO 45' '46 TO 48'
 '49 TO 51']

Columns: full_flat_type 
 Unique values: ['1 ROOM Improved' '2 ROOM 2-room' '2 ROOM DBSS' '2 ROOM Improved'
 '2 ROOM Model A' '2 ROOM Premium Apartment' '2 ROOM Standard'
 '3 ROOM DBSS' '3 ROOM Improved' '3 ROOM Model A' '3 ROOM New Generation'
 '3 ROOM Premium Apartment' '3 ROOM Simplified' '3 ROOM Standard'
 '3 ROOM Terrace' '4 ROOM Adjoined flat' '4 ROOM DBSS' '4 ROOM Improved'
 '4 ROOM Model A' '4 ROOM Model A2' '4 ROOM New Generation'
 '4 ROOM Premium Apartment' '4 ROOM Premium Apartment Loft'
 '4 ROOM Simplified' '4 ROOM Standard' '4 ROOM Terrace' '4 ROOM Type S1'
 '5 ROOM Adjoined flat' '5 ROOM DBSS' '5 ROOM Improved'
 '5 ROOM Improved-Maisonette' '5 ROOM Model A

**Comments**

- Storey-range and mid-storey data need to be revised
- No further issues in data types and data value range

## 1.2 Flat Storey:

In [10]:
#Checking storey_range data unique elements
np.sort(df['storey_range'].unique())

array(['01 TO 03', '01 TO 05', '04 TO 06', '06 TO 10', '07 TO 09',
       '10 TO 12', '11 TO 15', '13 TO 15', '16 TO 18', '16 TO 20',
       '19 TO 21', '21 TO 25', '22 TO 24', '25 TO 27', '26 TO 30',
       '28 TO 30', '31 TO 33', '31 TO 35', '34 TO 36', '36 TO 40',
       '37 TO 39', '40 TO 42', '43 TO 45', '46 TO 48', '49 TO 51'],
      dtype=object)

**Issue 1**
- There are overlapping Storey Range
- For the ease of subsequent analysis, we will re-label each unit into 'storey-range' based on their 'mid' value
- Example:
    - A unit with 'mid' value of 5 will belong to storey range '04 TO 06'
    
**Issue 2**<br>
It is not necessary to have multiple columns providing information on the unit's storey. As such we will be retaining only two revised columns 'storey-range' and 'mid'.

### 1.2.1 Addressing issues in columns related to Flat storeys

In [11]:
#Creating new list of unique storey_range elements
new_storey_range = []

for i in range (1,50,3):
    s = f'{i:02d} TO {i+2:02d}'
    new_storey_range.append(s)
new_storey_range

['01 TO 03',
 '04 TO 06',
 '07 TO 09',
 '10 TO 12',
 '13 TO 15',
 '16 TO 18',
 '19 TO 21',
 '22 TO 24',
 '25 TO 27',
 '28 TO 30',
 '31 TO 33',
 '34 TO 36',
 '37 TO 39',
 '40 TO 42',
 '43 TO 45',
 '46 TO 48',
 '49 TO 51']

In [12]:
# Round the numbers up to the nearest multiple of 3
df['upper'] = np.ceil(df['upper'] / 3) * 3

In [13]:
# Define the bins based on the intervals
bins = [int(i.split(' ')[0]) for i in new_storey_range] + [int(new_storey_range[-1].split(' ')[-1])+1]

# Define the labels based on the intervals
labels = new_storey_range

# Create a new column based on the bins and labels
df['storey_range'] = pd.cut(df['upper'], bins=bins, labels=labels)
# Convert column to 'string'
df['storey_range'] = df['storey_range'].astype(str)

In [14]:
#Check:
np.sort(df['storey_range'].unique())

array(['01 TO 03', '04 TO 06', '07 TO 09', '10 TO 12', '13 TO 15',
       '16 TO 18', '19 TO 21', '22 TO 24', '25 TO 27', '28 TO 30',
       '31 TO 33', '34 TO 36', '37 TO 39', '40 TO 42', '43 TO 45',
       '46 TO 48', '49 TO 51'], dtype=object)

In [15]:
# revise df['mid_storey'] values
df['mid_storey'] = df['upper']-1

In [16]:
#drop unnecasrry columns
df.drop(columns = ['lower','mid','upper'],inplace=True)


In [17]:
Flat_col.remove('lower')
Flat_col.remove('upper')
Flat_col.remove('mid')

df.loc[:5,Flat_col]

,storey_range,mid_storey,full_flat_type,flat_model,flat_type,floor_area_sqm,floor_area_sqft
0,10 TO 12,11.0,4 ROOM Model A,Model A,4 ROOM,90.0,968.760
1,07 TO 09,8.0,5 ROOM Improved,Improved,5 ROOM,130.0,1399.320
2,13 TO 15,14.0,EXECUTIVE Apartment,Apartment,EXECUTIVE,144.0,1550.016
3,04 TO 06,5.0,4 ROOM Model A,Model A,4 ROOM,103.0,1108.692
4,01 TO 03,2.0,4 ROOM Simplified,Simplified,4 ROOM,83.0,893.412
5,07 TO 09,8.0,3 ROOM Improved,Improved,3 ROOM,65.0,699.660


## 1.3 Dropping other redudant Flat feature columns

- flat_full_type
    - represented by flat_model and flat_type
- floor_area_sqm
    - similar to floor_area_sqft

In [18]:
df.drop(columns=['full_flat_type','floor_area_sqm'],inplace=True)
Flat_col.remove('full_flat_type')
Flat_col.remove('floor_area_sqm')

df.loc[:5,Flat_col]

,storey_range,mid_storey,flat_model,flat_type,floor_area_sqft
0,10 TO 12,11.0,Model A,4 ROOM,968.760
1,07 TO 09,8.0,Improved,5 ROOM,1399.320
2,13 TO 15,14.0,Apartment,EXECUTIVE,1550.016
3,04 TO 06,5.0,Model A,4 ROOM,1108.692
4,01 TO 03,2.0,Simplified,4 ROOM,893.412
5,07 TO 09,8.0,Improved,3 ROOM,699.660


## 1.4 Summary for this section:

Original number of features: 77<br>
Amendments:
- Removed 5 features relating to flat
- Revised columns 'storey range' and 'mid-storey'
- Resulting dataframe number of features: 72

In [19]:
df.loc[:,Flat_col].dtypes

storey_range        object
mid_storey         float64
flat_model          object
flat_type           object
floor_area_sqft    float64
dtype: object

In [20]:
df.shape

(150634, 72)

# Part 2: Features: Transaction <a class="anchor" id="Transaction"></a>

- Transaction period
- Transaction amount (resale price)

In [21]:
Transaction_col = [col for col in df.columns if (('Year' in col)or('Month'in col))]
Transaction_col.extend(['resale_price'])
Transaction_col

['Tranc_YearMonth', 'Tranc_Year', 'Tranc_Month', 'resale_price']

In [22]:
#lets take a look of transaction related columns
df.loc[:5,Transaction_col]

,Tranc_YearMonth,Tranc_Year,Tranc_Month,resale_price
0,2016-05,2016,5,680000.0
1,2012-07,2012,7,665000.0
2,2013-07,2013,7,838000.0
3,2012-04,2012,4,550000.0
4,2017-12,2017,12,298000.0
5,2013-01,2013,1,335000.0


## 2.1 Data types and Value check

In [23]:
df.loc[:,Transaction_col].dtypes

Tranc_YearMonth     object
Tranc_Year           int64
Tranc_Month          int64
resale_price       float64
dtype: object

In [24]:
get_min_max_df(df.loc[:,Transaction_col])

,Column Name,Minimum Value,Maximum Value
0,Tranc_Year,2012.0,2021.0
1,Tranc_Month,1.0,12.0
2,resale_price,150000.0,1258000.0


In [25]:
sorted(df['Tranc_YearMonth'].unique())

['2012-03',
 '2012-04',
 '2012-05',
 '2012-06',
 '2012-07',
 '2012-08',
 '2012-09',
 '2012-10',
 '2012-11',
 '2012-12',
 '2013-01',
 '2013-02',
 '2013-03',
 '2013-04',
 '2013-05',
 '2013-06',
 '2013-07',
 '2013-08',
 '2013-09',
 '2013-10',
 '2013-11',
 '2013-12',
 '2014-01',
 '2014-02',
 '2014-03',
 '2014-04',
 '2014-05',
 '2014-06',
 '2014-07',
 '2014-08',
 '2014-09',
 '2014-10',
 '2014-11',
 '2014-12',
 '2015-01',
 '2015-02',
 '2015-03',
 '2015-04',
 '2015-05',
 '2015-06',
 '2015-07',
 '2015-08',
 '2015-09',
 '2015-10',
 '2015-11',
 '2015-12',
 '2016-01',
 '2016-02',
 '2016-03',
 '2016-04',
 '2016-05',
 '2016-06',
 '2016-07',
 '2016-08',
 '2016-09',
 '2016-10',
 '2016-11',
 '2016-12',
 '2017-01',
 '2017-02',
 '2017-03',
 '2017-04',
 '2017-05',
 '2017-06',
 '2017-07',
 '2017-08',
 '2017-09',
 '2017-10',
 '2017-11',
 '2017-12',
 '2018-01',
 '2018-02',
 '2018-03',
 '2018-04',
 '2018-05',
 '2018-06',
 '2018-07',
 '2018-08',
 '2018-09',
 '2018-10',
 '2018-11',
 '2018-12',
 '2019-01',
 '20

**Comments**

- No issues in data types and data value range
- however we would want to convert Tranc_YearMonth to date_time format for analysis

## 2.2 Convert Tranc_YearMonth to date_time format

In [26]:
df['Tranc_YearMonth'] = pd.to_datetime(df['Tranc_YearMonth'])

In [27]:
df.loc[:5,Transaction_col].dtypes

Tranc_YearMonth    datetime64[ns]
Tranc_Year                  int64
Tranc_Month                 int64
resale_price              float64
dtype: object

## 2.3 Summary for this section:

Amendments:
- Converting Tranc_YearMonth to date_time format

Resulting dataframe number of features: 72 (*no change*)

# Part 3: Features: Block <a class="anchor" id="Block"></a>

Refering to the building features the flat is located in

In [28]:
Block_col = [col for col in df.columns if ('sold' in col) or ('rental' in col)]
Block_col.extend(['max_floor_lvl','total_dwelling_units','hdb_age','lease_commence_date','year_completed','residential'])
Block_col

['1room_sold',
 '2room_sold',
 '3room_sold',
 '4room_sold',
 '5room_sold',
 'exec_sold',
 'multigen_sold',
 'studio_apartment_sold',
 '1room_rental',
 '2room_rental',
 '3room_rental',
 'other_room_rental',
 'max_floor_lvl',
 'total_dwelling_units',
 'hdb_age',
 'lease_commence_date',
 'year_completed',
 'residential']

In [29]:
df.loc[:5,Block_col]

,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,max_floor_lvl,total_dwelling_units,hdb_age,lease_commence_date,year_completed,residential
0,0,0,0,96,46,0,0,0,0,0,0,0,25,142,15,2006,2005,Y
1,0,0,0,56,56,0,0,0,0,0,0,0,9,112,34,1987,1987,Y
2,0,0,0,0,30,60,0,0,0,0,0,0,16,90,24,1997,1996,Y
3,0,0,0,68,7,0,0,0,0,0,0,0,11,75,29,1992,1990,Y
4,0,0,0,30,18,0,0,0,0,0,0,0,4,48,34,1987,1987,Y
5,0,0,188,0,5,0,0,0,0,0,0,0,12,193,46,1975,1974,Y


## 3.1 Data types and Value check

In [30]:
df.loc[:,Block_col].dtypes

1room_sold                int64
2room_sold                int64
3room_sold                int64
4room_sold                int64
5room_sold                int64
exec_sold                 int64
multigen_sold             int64
studio_apartment_sold     int64
1room_rental              int64
2room_rental              int64
3room_rental              int64
other_room_rental         int64
max_floor_lvl             int64
total_dwelling_units      int64
hdb_age                   int64
lease_commence_date       int64
year_completed            int64
residential              object
dtype: object

In [31]:
df['residential'].unique()

array(['Y'], dtype=object)

In [32]:
get_min_max_df(df.loc[:,Block_col])

,Column Name,Minimum Value,Maximum Value
0,1room_sold,0,255
1,2room_sold,0,225
2,3room_sold,0,528
3,4room_sold,0,316
4,5room_sold,0,164
5,exec_sold,0,135
6,multigen_sold,0,66
7,studio_apartment_sold,0,142
8,1room_rental,0,110
9,2room_rental,0,452


**Comments**
- There is only one unique elements in residential - we shall drop this columns
- No further issues in data types and data value range

### 3.1.2 Dropping <code>residential</code> column

In [33]:
print(f"Number of columns before dropping: {len(df.columns)}")
df.drop(columns = 'residential', inplace = True)
print(f"Number of columns after dropping: {len(df.columns)}")


Number of columns before dropping: 72
Number of columns after dropping: 71


## 3.2 Summary for this section:
- Dropped one columns
- No. of columns now: 71

# Part 4: Features: Location <a class="anchor" id="Location"></a>

In [34]:
Location_col = ['Latitude','Longitude','planning_area','town','block','street_name','address','postal']

In [35]:
df.loc[:5,Location_col]

,Latitude,Longitude,planning_area,town,block,street_name,address,postal
0,1.314299,103.872828,Kallang,KALLANG/WHAMPOA,3B,UPP BOON KENG RD,"3B, UPP BOON KENG RD",382003
1,1.346086,103.855078,Bishan,BISHAN,153,BISHAN ST 13,"153, BISHAN ST 13",570153
2,1.343867,103.760535,Bukit Batok,BUKIT BATOK,289B,BT BATOK ST 25,"289B, BT BATOK ST 25",651289
3,1.358245,103.845504,Bishan,BISHAN,232,BISHAN ST 22,"232, BISHAN ST 22",570232
4,1.414745,103.835532,Yishun,YISHUN,876,YISHUN ST 81,"876, YISHUN ST 81",760876
5,1.321703,103.886196,Geylang,GEYLANG,95,ALJUNIED CRES,"95, ALJUNIED CRES",380095


## 4.1 Data types and Value check

In [36]:
df.loc[:,Location_col].dtypes

Latitude         float64
Longitude        float64
planning_area     object
town              object
block             object
street_name       object
address           object
postal            object
dtype: object

In [37]:
get_min_max_df(df.loc[:,Location_col])

,Column Name,Minimum Value,Maximum Value
0,Latitude,1.270380,1.457071
1,Longitude,103.685228,103.987804


In [38]:
_ls = list(df.loc[:,Location_col].select_dtypes(include=['object']).columns)
for i in _ls:
    _a = np.sort(df[i].unique())
    print(f"Columns: {i} \n Unique values: {_a}")

Columns: planning_area 
 Unique values: ['Ang Mo Kio' 'Bedok' 'Bishan' 'Bukit Batok' 'Bukit Merah' 'Bukit Panjang'
 'Bukit Timah' 'Changi' 'Choa Chu Kang' 'Clementi' 'Downtown Core'
 'Geylang' 'Hougang' 'Jurong East' 'Jurong West' 'Kallang' 'Marine Parade'
 'Novena' 'Outram' 'Pasir Ris' 'Punggol' 'Queenstown' 'Rochor' 'Sembawang'
 'Sengkang' 'Serangoon' 'Tampines' 'Tanglin' 'Toa Payoh'
 'Western Water Catchment' 'Woodlands' 'Yishun']
Columns: town 
 Unique values: ['ANG MO KIO' 'BEDOK' 'BISHAN' 'BUKIT BATOK' 'BUKIT MERAH' 'BUKIT PANJANG'
 'BUKIT TIMAH' 'CENTRAL AREA' 'CHOA CHU KANG' 'CLEMENTI' 'GEYLANG'
 'HOUGANG' 'JURONG EAST' 'JURONG WEST' 'KALLANG/WHAMPOA' 'MARINE PARADE'
 'PASIR RIS' 'PUNGGOL' 'QUEENSTOWN' 'SEMBAWANG' 'SENGKANG' 'SERANGOON'
 'TAMPINES' 'TOA PAYOH' 'WOODLANDS' 'YISHUN']
Columns: block 
 Unique values: ['1' '10' '100' ... '99C' '9A' '9B']
Columns: street_name 
 Unique values: ['ADMIRALTY DR' 'ADMIRALTY LINK' 'AH HOOD RD' 'ALJUNIED CRES'
 'ALJUNIED RD' 'ANCHORVALE CRE

**Comments**

- <code>postal</code> should be numerical instead of object
- noted that <code>postal</code> contained non-numerical elements
- No other issues with data types and data values

## 4.2 Addressing <code>postal</code> column

In [39]:
#looking at what is non numerical element in this column
df.loc[~df['postal'].str.isdigit(),'postal'].unique()

array(['NIL'], dtype=object)

**Issue**

- 'postal' (referring to postal code)  has some missing data (NIL)
- Since there is other columns providing much information on the location of the HDB building, this column 'postal' shall be dropped

In [40]:
df.drop(columns='postal',inplace=True)

In [41]:
Location_col.remove('postal')

In [42]:
df.shape

(150634, 70)

## 4.3 Summary for this section:

Dropped 1 column

Overall resulting number of columns in df: 70

# Part 5: Features: Facilities <a class="anchor" id="Facilities"></a>

#### *Rename columns for easier identification school-related features*

In [43]:
df.rename(columns = {'vacancy':'pri_sch_vacancy',
                  'affiliation':'sec_affiliation_pri_sch',
                  'pri_sch_affiliation':'pri_affiliation_sec_sch',
                  'cutoff_point':'sec_sch_cutoffpoints'},inplace=True)

## 5.1 Addressing columns with missing data

In [44]:
#Recall from earlier we found that mall and hawker related columns contain missing data
df_missing(df)

,col,num_nulls,perc_null
0,Mall_Nearest_Distance,829,0.01
1,Mall_Within_500m,92789,0.62
2,Mall_Within_1km,25426,0.17
3,Mall_Within_2km,1940,0.01
4,Hawker_Within_500m,97390,0.65
5,Hawker_Within_1km,60868,0.40
6,Hawker_Within_2km,29202,0.19


In [45]:
hwker_mall_col = [col for col in df.columns if ('Mall' in col) | ('Hawker' in col)]
hwker_mall_col

['Mall_Nearest_Distance',
 'Mall_Within_500m',
 'Mall_Within_1km',
 'Mall_Within_2km',
 'Hawker_Nearest_Distance',
 'Hawker_Within_500m',
 'Hawker_Within_1km',
 'Hawker_Within_2km']

In [46]:
get_min_max_df(df.loc[:,hwker_mall_col])

,Column Name,Minimum Value,Maximum Value
0,Mall_Nearest_Distance,0.000000,3496.402910
1,Mall_Within_500m,1.000000,6.000000
2,Mall_Within_1km,1.000000,15.000000
3,Mall_Within_2km,1.000000,43.000000
4,Hawker_Nearest_Distance,1.873295,4907.036414
5,Hawker_Within_500m,1.000000,5.000000
6,Hawker_Within_1km,1.000000,9.000000
7,Hawker_Within_2km,1.000000,19.000000


**Comments**

- Columns that have missing data are Hawker or Mall related features
- Looking at the min and max values, we suspect the reasons for Missing data include:
    - for Mall or Hawker within 500m, 1km, 2km columns --> NaN is likely because there is **no** Mall/ Hawker in that specified distance (the min value is 1 instead of 0)
    - for Mall_Nearest_Distance --> NaN is likely because the record does not include distances if it is beyond 3.5km
- **Plan**: we shall analyse these columns with Missing Data further and address the NaN appropriately

### 5.1.1 Missing Data in Mall witin 500m,1km,2km columns<br> (with no NaN in Mall_Nearest_Distance)

In [47]:
mall_col = [col for col in df.columns if 'Mall' in col]
mall_col

['Mall_Nearest_Distance',
 'Mall_Within_500m',
 'Mall_Within_1km',
 'Mall_Within_2km']

#### Missing in Mall_Within_500m

In [48]:
row_filter = df['Mall_Nearest_Distance'].notnull() & df['Mall_Within_500m'].isnull()
df.loc[row_filter,mall_col]

,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km
0,1094.090418,NaN,NaN,7.0
1,866.941448,NaN,1.0,3.0
2,1459.579948,NaN,NaN,4.0
3,950.175199,NaN,1.0,4.0
4,729.771895,NaN,1.0,2.0
...,...,...,...,...
150626,1216.129901,NaN,NaN,6.0
150628,565.636046,NaN,1.0,3.0
150629,585.138715,NaN,4.0,6.0
150631,1790.053482,NaN,NaN,2.0


In [49]:
df.loc[row_filter,'Mall_Nearest_Distance'].min()

500.0555752

**Comment**<br>
These 91960 units have NaN in 'Mall_Within_500m' column because they do not have any malls within 500m.
- Plan: Fill these row's 'Mall_Within_500m' column with Zero

In [50]:
row_filter = df['Mall_Nearest_Distance'].notnull() & df['Mall_Within_500m'].isnull()
df.loc[row_filter,'Mall_Within_500m']= df.loc[row_filter,'Mall_Within_500m'].fillna(0)

#### Missing in Mall_Within_1Km

In [51]:
row_filter = df['Mall_Nearest_Distance'].notnull() & df['Mall_Within_1km'].isnull()
df.loc[row_filter,mall_col]

,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km
0,1094.090418,0.0,NaN,7.0
2,1459.579948,0.0,NaN,4.0
7,1749.147519,0.0,NaN,3.0
27,1063.974768,0.0,NaN,4.0
35,1211.930580,0.0,NaN,6.0
...,...,...,...,...
150600,1036.008434,0.0,NaN,3.0
150606,1019.573002,0.0,NaN,4.0
150613,1051.126583,0.0,NaN,6.0
150626,1216.129901,0.0,NaN,6.0


In [52]:
df.loc[row_filter,'Mall_Nearest_Distance'].min()

1000.023299

**Comment**<br>
These 24597 units have NaN in 'Mall_Within_1km' column because they do not have any malls within 1km.
- Plan: Fill these row's 'Mall_Within_1km' column with Zero

In [53]:
row_filter = df['Mall_Nearest_Distance'].notnull() & df['Mall_Within_1km'].isnull()
df.loc[row_filter,'Mall_Within_1km']= df.loc[row_filter,'Mall_Within_1km'].fillna(0)

#### Missing in Mall_Within_2km

In [54]:
row_filter = df['Mall_Nearest_Distance'].notnull() & df['Mall_Within_2km'].isnull()
df.loc[row_filter,mall_col]

,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km
109,2050.201309,0.0,0.0,NaN
220,2247.327448,0.0,0.0,NaN
662,2218.607002,0.0,0.0,NaN
672,2038.439070,0.0,0.0,NaN
798,2115.704584,0.0,0.0,NaN
...,...,...,...,...
149097,2103.873021,0.0,0.0,NaN
149446,2218.607002,0.0,0.0,NaN
149450,2260.225191,0.0,0.0,NaN
150095,2007.777071,0.0,0.0,NaN


In [55]:
df.loc[row_filter,'Mall_Nearest_Distance'].min()

2000.175732

**Comment**<br>
These 1111  units have NaN in 'Mall_Within_2km' column because they do not have any malls within 2km.
- Plan: Fill these row's 'Mall_Within_2km' column with Zero

In [56]:
row_filter = df['Mall_Nearest_Distance'].notnull() & df['Mall_Within_2km'].isnull()
df.loc[row_filter,'Mall_Within_2km']= df.loc[row_filter,'Mall_Within_2km'].fillna(0)

### 5.1.2 Missing in 'Mall_Nearest_Distance'

In [57]:
row_filter = df['Mall_Nearest_Distance'].isnull()
df.loc[row_filter,mall_col]

,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km
75,NaN,NaN,NaN,NaN
321,NaN,NaN,NaN,NaN
478,NaN,NaN,NaN,NaN
643,NaN,NaN,NaN,NaN
691,NaN,NaN,NaN,NaN
...,...,...,...,...
150296,NaN,NaN,NaN,NaN
150388,NaN,NaN,NaN,NaN
150394,NaN,NaN,NaN,NaN
150479,NaN,NaN,NaN,NaN


In [58]:
#Showing Missing data summary for data rows who has NaN in Mall_Nearest_Distance
row_filter = df['Mall_Nearest_Distance'].isnull()
df_missing(df.loc[row_filter,mall_col])

,col,num_nulls,perc_null
0,Mall_Nearest_Distance,829,1.0
1,Mall_Within_500m,829,1.0
2,Mall_Within_1km,829,1.0
3,Mall_Within_2km,829,1.0


In [59]:
#Showing Missing data summary for data rows who DO NOT have NaN in Mall_Nearest_Distance
row_filter = df['Mall_Nearest_Distance'].notnull()
df_missing(df.loc[row_filter,mall_col])

--No Missing Data--


In [60]:
df.loc[row_filter,mall_col]

,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km
0,1094.090418,0.0,0.0,7.0
1,866.941448,0.0,1.0,3.0
2,1459.579948,0.0,0.0,4.0
3,950.175199,0.0,1.0,4.0
4,729.771895,0.0,1.0,2.0
...,...,...,...,...
150629,585.138715,0.0,4.0,6.0
150630,250.084466,1.0,1.0,4.0
150631,1790.053482,0.0,0.0,2.0
150632,587.244922,0.0,3.0,3.0


In [61]:
#Showing Missing data summary for data rows REGARDLESS if there is NaN in Mall_Nearest_Distance
df_missing(df.loc[:,mall_col])

,col,num_nulls,perc_null
0,Mall_Nearest_Distance,829,0.01
1,Mall_Within_500m,829,0.01
2,Mall_Within_1km,829,0.01
3,Mall_Within_2km,829,0.01


**Comment**<br>
These 829  units have NaN in 'Mall_Nearest_Distance' column also have missing data in Mall within 500m/1km/2km columns.
- As such:
    - Plan: Fill these row's Mall within 500m/1km/2km columns with Zero

In [62]:
columns = ['Mall_Within_500m', 'Mall_Within_1km', 'Mall_Within_2km']
df.loc[:,columns] = df.loc[:,columns].fillna(0)

**Comment**:<br>
There is still 829 flats with no recorded Mall_Nearest_Distance as distance beyond 3.5km was not recorded.

To address these missing data, we will assume that distance is within 4km:
- Generally, HDB flats are located with transportation facilities such as MRT stations within reasonable distance
- Most MRT stations have malls (around [100](https://mallspaces.asia/resources/mrt/ultimate-malls-list-with-mrt/) out of the [130+ stations](https://en.wikipedia.org/wiki/Mass_Rapid_Transit_(Singapore).

As such, we will fill NaN with 4000 meters for Mall_Nearest_Distance.

In [63]:
df['Mall_Nearest_Distance']=df['Mall_Nearest_Distance'].fillna(4000)

### 5.1.3 Missing 'Hawker' Data

In [64]:
hwkr_col = [col for col in df.columns if 'Hawker' in col]
hwkr_col

['Hawker_Nearest_Distance',
 'Hawker_Within_500m',
 'Hawker_Within_1km',
 'Hawker_Within_2km']

#### Missing in Hawker_Within_500m

In [65]:
row_filter = df['Hawker_Within_500m'].isnull()
df.loc[row_filter,hwkr_col].isnull().sum()
df.loc[row_filter,hwkr_col]

Hawker_Nearest_Distance        0
Hawker_Within_500m         97390
Hawker_Within_1km          60868
Hawker_Within_2km          29202
dtype: int64

,Hawker_Nearest_Distance,Hawker_Within_500m,Hawker_Within_1km,Hawker_Within_2km
1,640.151925,NaN,1.0,7.0
2,1762.082341,NaN,NaN,1.0
3,726.215262,NaN,1.0,9.0
4,1540.151439,NaN,NaN,1.0
6,516.981824,NaN,1.0,4.0
...,...,...,...,...
150628,2443.370839,NaN,NaN,NaN
150629,973.725973,NaN,1.0,3.0
150630,1807.769865,NaN,NaN,1.0
150631,1076.260436,NaN,NaN,4.0


In [66]:
df.loc[row_filter,'Hawker_Nearest_Distance'].min()

500.2087255

**Comment**<br>
These 97390 units have NaN in 'Hawker_Within_500m' column because they do not have any hawkers within 500m.
- Plan: Fill these row's 'Hawker_Within_500m' column with Zero

In [67]:
df.loc[row_filter,'Hawker_Within_500m']= df.loc[row_filter,'Hawker_Within_500m'].fillna(0)

#### Missing in Hawker_Within_1km

In [68]:
row_filter = df['Hawker_Within_1km'].isnull()
df.loc[row_filter,hwkr_col].isnull().sum()
df.loc[row_filter,hwkr_col]

Hawker_Nearest_Distance        0
Hawker_Within_500m             0
Hawker_Within_1km          60868
Hawker_Within_2km          29202
dtype: int64

,Hawker_Nearest_Distance,Hawker_Within_500m,Hawker_Within_1km,Hawker_Within_2km
2,1762.082341,0.0,NaN,1.0
4,1540.151439,0.0,NaN,1.0
8,1032.848455,0.0,NaN,4.0
9,1743.704293,0.0,NaN,1.0
10,1634.750527,0.0,NaN,1.0
...,...,...,...,...
150625,4491.675680,0.0,NaN,NaN
150627,1957.044643,0.0,NaN,1.0
150628,2443.370839,0.0,NaN,NaN
150630,1807.769865,0.0,NaN,1.0


In [69]:
df.loc[row_filter,'Hawker_Nearest_Distance'].min()

1000.074344

**Comment**<br>
These 60868  units have NaN in 'Hawker_Within_1km' column because they do not have any hawkers within 1km.
- Plan: Fill these row's 'Hawker_Within_500m' column with Zero

In [70]:
df.loc[row_filter,'Hawker_Within_1km']= df.loc[row_filter,'Hawker_Within_1km'].fillna(0)

#### Missing in Hawker_Within_2km

In [71]:
row_filter = df['Hawker_Within_2km'].isnull()
df.loc[row_filter,hwkr_col].isnull().sum()
df.loc[row_filter,hwkr_col]

Hawker_Nearest_Distance        0
Hawker_Within_500m             0
Hawker_Within_1km              0
Hawker_Within_2km          29202
dtype: int64

,Hawker_Nearest_Distance,Hawker_Within_500m,Hawker_Within_1km,Hawker_Within_2km
26,2028.864896,0.0,0.0,NaN
27,2044.044484,0.0,0.0,NaN
30,3472.690804,0.0,0.0,NaN
31,2278.363660,0.0,0.0,NaN
33,3159.026726,0.0,0.0,NaN
...,...,...,...,...
150615,4612.654746,0.0,0.0,NaN
150620,3794.986446,0.0,0.0,NaN
150623,2537.537184,0.0,0.0,NaN
150625,4491.675680,0.0,0.0,NaN


In [72]:
df.loc[row_filter,'Hawker_Nearest_Distance'].min()

2000.871907

**Comment**<br>
These 29202 units have NaN in 'Hawker_Within_2km' column because they do not have any hawkers within 2km.
- Plan: Fill these row's 'Hawker_Within_500m' column with Zero

In [73]:
df.loc[row_filter,'Hawker_Within_2km']= df.loc[row_filter,'Hawker_Within_2km'].fillna(0)

### Check if all missing data is address:

In [74]:
df_missing(df)

--No Missing Data--


In [75]:
df.shape

(150634, 70)

### Summary for this section:
At this point, we have completed issues with missing data
- Fill  NA with appropriate values

## 5.2 Boolean columns
> In this segment, we are ensuring Boolean columns have values as '0' and '1'.

|Boolean Col|Description|
|---|---|
|commercial| boolean value if resale flat has commercial units in the same block|
|market_hawker| boolean value if resale flat has a market or hawker centre in the same block|
|multistorey_carpark| boolean value if resale flat has a multistorey carpark in the same block|
|precinct_pavilion| boolean value if resale flat has a pavilion in the same block|
|bus_interchange| boolean value if the nearest MRT station is also a bus interchange|
|mrt_interchange| boolean value if the nearest MRT station is a train interchange station|
|pri_affiliation_sec_sch| boolean value if the nearest primary school has a secondary school affiliation|
|sec_affiliation_pri_sch| boolean value if the nearest secondary school has an primary school affiliation|

In [76]:
Bool=[
    'commercial',
    'market_hawker',
    'multistorey_carpark',
    'precinct_pavilion',
    'bus_interchange',
    'mrt_interchange',
    'pri_affiliation_sec_sch',
    'sec_affiliation_pri_sch']

In [77]:
df[Bool].dtypes

commercial                 object
market_hawker              object
multistorey_carpark        object
precinct_pavilion          object
bus_interchange             int64
mrt_interchange             int64
pri_affiliation_sec_sch     int64
sec_affiliation_pri_sch     int64
dtype: object

In [78]:
for bool_col in Bool:
    print(f"\nColumn '{bool_col}',\nUnique elements are\n {df[bool_col].unique()}")


Column 'commercial',
Unique elements are
 ['N' 'Y']

Column 'market_hawker',
Unique elements are
 ['N' 'Y']

Column 'multistorey_carpark',
Unique elements are
 ['N' 'Y']

Column 'precinct_pavilion',
Unique elements are
 ['N' 'Y']

Column 'bus_interchange',
Unique elements are
 [0 1]

Column 'mrt_interchange',
Unique elements are
 [0 1]

Column 'pri_affiliation_sec_sch',
Unique elements are
 [1 0]

Column 'sec_affiliation_pri_sch',
Unique elements are
 [0 1]


In [79]:
df.loc[:5,Bool]

,commercial,market_hawker,multistorey_carpark,precinct_pavilion,bus_interchange,mrt_interchange,pri_affiliation_sec_sch,sec_affiliation_pri_sch
0,N,N,N,N,0,0,1,0
1,N,N,N,N,1,1,1,0
2,N,N,N,N,1,0,0,0
3,Y,N,N,N,1,1,1,1
4,N,N,N,N,0,0,0,0
5,Y,N,N,N,0,1,1,0


In [80]:
#Covert all 'Y' and 'N' to numerical values
_l = df[Bool].select_dtypes(include=['object']).columns.tolist()
for col in _l:
    df[col] = df[col].map({'Y': 1, 'N': 0})

In [81]:
df.loc[:5,Bool]

,commercial,market_hawker,multistorey_carpark,precinct_pavilion,bus_interchange,mrt_interchange,pri_affiliation_sec_sch,sec_affiliation_pri_sch
0,0,0,0,0,0,0,1,0
1,0,0,0,0,1,1,1,0
2,0,0,0,0,1,0,0,0
3,1,0,0,0,1,1,1,1
4,0,0,0,0,0,0,0,0
5,1,0,0,0,0,1,1,0


In [82]:
df[Bool].dtypes

commercial                 int64
market_hawker              int64
multistorey_carpark        int64
precinct_pavilion          int64
bus_interchange            int64
mrt_interchange            int64
pri_affiliation_sec_sch    int64
sec_affiliation_pri_sch    int64
dtype: object

**Comment**

Rename Bool columns for clarity

In [83]:
for col_name in Bool:
    new_name = 'Have_'+col_name
    df.rename(columns={col_name:new_name},inplace=True)

In [84]:
#original names
Bool

['commercial',
 'market_hawker',
 'multistorey_carpark',
 'precinct_pavilion',
 'bus_interchange',
 'mrt_interchange',
 'pri_affiliation_sec_sch',
 'sec_affiliation_pri_sch']

In [85]:
#New boolean column names
[col for col in df.columns if 'Have' in col]

['Have_commercial',
 'Have_market_hawker',
 'Have_multistorey_carpark',
 'Have_precinct_pavilion',
 'Have_bus_interchange',
 'Have_mrt_interchange',
 'Have_pri_affiliation_sec_sch',
 'Have_sec_affiliation_pri_sch']

### Summary for this section:
- Converted boolean columns to intergers
- Rename column names to include 'Have_' to indicate boolean columns
- There is now total 8 Boolean columns

## 5.3 Hawker, Commercial Units, Mall

In [86]:
shop_and_food = [col for col in df.columns if ('awker'in col) or ('Mall' in col) or ('commercial' in col)]
shop_and_food

['Have_commercial',
 'Have_market_hawker',
 'Mall_Nearest_Distance',
 'Mall_Within_500m',
 'Mall_Within_1km',
 'Mall_Within_2km',
 'Hawker_Nearest_Distance',
 'Hawker_Within_500m',
 'Hawker_Within_1km',
 'Hawker_Within_2km',
 'hawker_food_stalls',
 'hawker_market_stalls']

In [87]:
df.loc[:5,shop_and_food]

,Have_commercial,Have_market_hawker,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km,Hawker_Nearest_Distance,Hawker_Within_500m,Hawker_Within_1km,Hawker_Within_2km,hawker_food_stalls,hawker_market_stalls
0,0,0,1094.090418,0.0,0.0,7.0,154.753357,1.0,3.0,13.0,84,60
1,0,0,866.941448,0.0,1.0,3.0,640.151925,0.0,1.0,7.0,80,77
2,0,0,1459.579948,0.0,0.0,4.0,1762.082341,0.0,0.0,1.0,84,95
3,1,0,950.175199,0.0,1.0,4.0,726.215262,0.0,1.0,9.0,32,86
4,0,0,729.771895,0.0,1.0,2.0,1540.151439,0.0,0.0,1.0,45,0
5,1,0,684.004001,0.0,3.0,6.0,148.418247,2.0,5.0,11.0,79,82


### Data types and Value check

In [88]:
df.loc[:,shop_and_food].dtypes

Have_commercial              int64
Have_market_hawker           int64
Mall_Nearest_Distance      float64
Mall_Within_500m           float64
Mall_Within_1km            float64
Mall_Within_2km            float64
Hawker_Nearest_Distance    float64
Hawker_Within_500m         float64
Hawker_Within_1km          float64
Hawker_Within_2km          float64
hawker_food_stalls           int64
hawker_market_stalls         int64
dtype: object

In [89]:
get_min_max_df(df.loc[:,shop_and_food])

,Column Name,Minimum Value,Maximum Value
0,Have_commercial,0.000000,1.000000
1,Have_market_hawker,0.000000,1.000000
2,Mall_Nearest_Distance,0.000000,4000.000000
3,Mall_Within_500m,0.000000,6.000000
4,Mall_Within_1km,0.000000,15.000000
5,Mall_Within_2km,0.000000,43.000000
6,Hawker_Nearest_Distance,1.873295,4907.036414
7,Hawker_Within_500m,0.000000,5.000000
8,Hawker_Within_1km,0.000000,9.000000
9,Hawker_Within_2km,0.000000,19.000000


**Comments**

- No issues in data types and data value range

## 5.4 School

In [90]:
Sch_col = [col for col in df.columns if 'sch' in col]
Sch_col

['pri_sch_nearest_distance',
 'pri_sch_name',
 'pri_sch_vacancy',
 'Have_pri_affiliation_sec_sch',
 'pri_sch_latitude',
 'pri_sch_longitude',
 'sec_sch_nearest_dist',
 'sec_sch_name',
 'sec_sch_cutoffpoints',
 'Have_sec_affiliation_pri_sch',
 'sec_sch_latitude',
 'sec_sch_longitude']

In [91]:
df.loc[:5,Sch_col]

,pri_sch_nearest_distance,pri_sch_name,pri_sch_vacancy,Have_pri_affiliation_sec_sch,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,sec_sch_cutoffpoints,Have_sec_affiliation_pri_sch,sec_sch_latitude,sec_sch_longitude
0,1138.633422,Geylang Methodist School,78,1,1.317659,103.882504,1138.633422,Geylang Methodist School,224,0,1.317659,103.882504
1,415.607357,Kuo Chuan Presbyterian Primary School,45,1,1.349783,103.854529,447.894399,Kuo Chuan Presbyterian Secondary School,232,0,1.350110,103.854892
2,498.849039,Keming Primary School,39,0,1.345245,103.756265,180.074558,Yusof Ishak Secondary School,188,0,1.342334,103.760013
3,389.515528,Catholic High School,20,1,1.354789,103.844934,389.515528,Catholic High School,253,1,1.354789,103.844934
4,401.200584,Naval Base Primary School,74,0,1.416280,103.838798,312.025435,Orchid Park Secondary School,208,0,1.414888,103.838335
5,594.128568,Saint Margaret's Primary School,65,1,1.324470,103.881623,608.838174,Geylang Methodist School,224,0,1.317659,103.882504


### Data types and Value check

In [92]:
df.loc[:,Sch_col].dtypes

pri_sch_nearest_distance        float64
pri_sch_name                     object
pri_sch_vacancy                   int64
Have_pri_affiliation_sec_sch      int64
pri_sch_latitude                float64
pri_sch_longitude               float64
sec_sch_nearest_dist            float64
sec_sch_name                     object
sec_sch_cutoffpoints              int64
Have_sec_affiliation_pri_sch      int64
sec_sch_latitude                float64
sec_sch_longitude               float64
dtype: object

In [93]:
get_min_max_df(df.loc[:,Sch_col])

,Column Name,Minimum Value,Maximum Value
0,pri_sch_nearest_distance,45.668324,3305.841039
1,pri_sch_vacancy,20.000000,110.000000
2,Have_pri_affiliation_sec_sch,0.000000,1.000000
3,pri_sch_latitude,1.274962,1.456667
4,pri_sch_longitude,103.687724,103.962919
5,sec_sch_nearest_dist,38.913475,3638.977233
6,sec_sch_cutoffpoints,188.000000,260.000000
7,Have_sec_affiliation_pri_sch,0.000000,1.000000
8,sec_sch_latitude,1.276029,1.453280
9,sec_sch_longitude,103.687207,103.961105


In [94]:
_ls = list(df.loc[:,Sch_col].select_dtypes(include=['object']).columns)
for i in _ls:
    _a = np.sort(df[i].unique())
    print(f"Columns: {i} \n Unique values: {_a}")

Columns: pri_sch_name 
 Unique values: ['Admiralty Primary School' 'Ahmad Ibrahim Primary School'
 'Ai Tong School' 'Alexandra Primary School' 'Anchor Green Primary School'
 'Anderson Primary School' 'Ang Mo Kio Primary School'
 'Angsana Primary School' 'Beacon Primary School'
 'Bedok Green Primary School' 'Bendemeer Primary School'
 'Blangah Rise Primary School' 'Boon Lay Garden Primary School'
 'Bukit Panjang Primary School' 'Bukit View Primary School' 'CHIJ'
 'CHIJ Our Lady Queen of Peace' 'CHIJ Our Lady of Good Counsel'
 'CHIJ Our Lady of The Nativity' 'CHIJ Primary'
 "CHIJ Saint Nicholas Girls' School" 'Canberra Primary School'
 'Canossa Catholic Primary School' 'Cantonment Primary School'
 'Casuarina Primary School' 'Catholic High School' 'Cedar Primary School'
 'Changkat Primary School' 'Chongfu School' 'Chongzheng Primary School'
 'Chua Chu Kang Primary School' 'Clementi Primary School'
 'Compassvale Primary School' 'Concord Primary School'
 'Corporation Primary School' 'Damai 

**Comments**

- No issues in data types and data value range

## 5.5 Transport

In [95]:
txport = [col for col in df.columns if ('mrt'in col) or ('bus' in col)]
txport

['mrt_nearest_distance',
 'mrt_name',
 'Have_bus_interchange',
 'Have_mrt_interchange',
 'mrt_latitude',
 'mrt_longitude',
 'bus_stop_nearest_distance',
 'bus_stop_name',
 'bus_stop_latitude',
 'bus_stop_longitude']

In [96]:
df.loc[:5,txport]

,mrt_nearest_distance,mrt_name,Have_bus_interchange,Have_mrt_interchange,mrt_latitude,mrt_longitude,bus_stop_nearest_distance,bus_stop_name,bus_stop_latitude,bus_stop_longitude
0,330.083069,Kallang,0,0,1.311540,103.871731,29.427395,Blk 3B,1.314433,103.872600
1,903.659703,Bishan,1,1,1.350580,103.848305,58.207761,BLK 151A MKT,1.345659,103.855381
2,1334.251197,Bukit Batok,1,0,1.349561,103.749970,214.747860,Blk 289E,1.344064,103.758613
3,907.453484,Bishan,1,1,1.350580,103.848305,43.396521,Opp Bishan Nth Shop Mall,1.358045,103.845169
4,412.343032,Khatib,0,0,1.417131,103.832692,129.422752,Blk 873,1.415424,103.836477
5,678.291272,MacPherson,0,1,1.325859,103.890663,229.680469,Blk 90,1.323622,103.886962


### Data types and Value check

In [97]:
df.loc[:,txport].dtypes

mrt_nearest_distance         float64
mrt_name                      object
Have_bus_interchange           int64
Have_mrt_interchange           int64
mrt_latitude                 float64
mrt_longitude                float64
bus_stop_nearest_distance    float64
bus_stop_name                 object
bus_stop_latitude            float64
bus_stop_longitude           float64
dtype: object

In [98]:
get_min_max_df(df.loc[:,txport])

,Column Name,Minimum Value,Maximum Value
0,mrt_nearest_distance,21.971410,3544.504228
1,Have_bus_interchange,0.000000,1.000000
2,Have_mrt_interchange,0.000000,1.000000
3,mrt_latitude,1.265982,1.449057
4,mrt_longitude,103.697143,103.987305
5,bus_stop_nearest_distance,9.001285,443.964584
6,bus_stop_latitude,1.270759,1.456194
7,bus_stop_longitude,103.684206,103.987343


In [99]:
_ls = list(df.loc[:,txport].select_dtypes(include=['object']).columns)
for i in _ls:
    _a = np.sort(df[i].unique())
    print(f"Columns: {i} \n Unique values: {_a}")

Columns: mrt_name 
 Unique values: ['Admiralty' 'Aljunied' 'Ang Mo Kio' 'Bartley' 'Beauty World' 'Bedok'
 'Bedok North' 'Bedok Reservoir' 'Bencoolen' 'Bendemeer' 'Bishan'
 'Boon Keng' 'Boon Lay' 'Botanic Gardens' 'Braddell' 'Bras Basah'
 'Buangkok' 'Bugis' 'Bukit Batok' 'Bukit Gombak' 'Bukit Panjang'
 'Buona Vista' 'Caldecott' 'Canberra' 'Cashew' 'Changi Airport'
 'Chinatown' 'Chinese Garden' 'Choa Chu Kang' 'Clarke Quay' 'Clementi'
 'Commonwealth' 'Dakota' 'Dover' 'Eunos' 'Farrer Park' 'Farrer Road'
 'Geylang Bahru' 'HarbourFront' 'Hillview' 'Holland Village' 'Hougang'
 'Jalan Besar' 'Jurong East' 'Kaki Bukit' 'Kallang' 'Kembangan' 'Khatib'
 'Kovan' 'Labrador Park' 'Lakeside' 'Lavender' 'Little India'
 'Lorong Chuan' 'MacPherson' 'Marsiling' 'Marymount' 'Mattar'
 'Mountbatten' 'Nicoll Highway' 'Novena' 'Outram Park' 'Pasir Ris'
 'Paya Lebar' 'Pioneer' 'Potong Pasir' 'Punggol' 'Queenstown' 'Redhill'
 'Rochor' 'Sembawang' 'Sengkang' 'Serangoon' 'Simei' 'Tai Seng' 'Tampines'
 'Tampines E

**Comments**

- No issues in data types and data value range

## 5.6 Other Facilities

In [100]:
C_P_df = ['Have_multistorey_carpark','Have_precinct_pavilion']
C_P_df

['Have_multistorey_carpark', 'Have_precinct_pavilion']

### Data types and Value check

In [101]:
df.loc[:,C_P_df].dtypes

Have_multistorey_carpark    int64
Have_precinct_pavilion      int64
dtype: object

In [102]:
get_min_max_df(df.loc[:,C_P_df])

,Column Name,Minimum Value,Maximum Value
0,Have_multistorey_carpark,0,1
1,Have_precinct_pavilion,0,1


**Comments**

- No issues in data types and data value range

# Summary and Export:<a class="anchor" id="summary"></a>

Let's take a look at our resulting dataframe

In [103]:
# Columns that has not yet been addressed
[x for x in df.columns.tolist() if x not in Flat_col+Transaction_col+Block_col+Location_col+shop_and_food+Sch_col+txport+C_P_df]

['id']

In [104]:
df_summary(df)

Shape(col,rows): (150634, 70)
Number of duplicates: 0
------------------------------------------------------------
Number of each unqiue datatypes:
int64             32
float64           25
object            12
datetime64[ns]     1
dtype: int64
------------------------------------------------------------
Columns with missing values:
--No Missing Data--


**Summary**

We have:
- Addressed missing data
- Checked and ensure appropriate data values and data types
- Dropped some columns (resulting in 70 number of features)

**Now What?**
- Export Data
- Move on to Exploratory Data Analysis in next notebook

In [105]:
#Export data
df.to_csv("../data_output/cleaned_data.csv",index=False)